In [1]:
import pandas as pd
news = pd.read_csv(r'D:\Downloads\cleaned_data_med_amazo.csv')

In [2]:
news.head()

,ID,dates,name,ratings,reviews
0,4,2023-01-31,Ginkgo,2.7,No effect on tinnitus
1,5,2022-03-11,Ginkgo,4.3,"Take gingo for the pain in my legs, it works. ..."
2,6,2021-10-07,Ginkgo,3.0,Besides the fact that individual effects invar...
3,7,2021-09-01,Ginkgo,2.0,About a week after I started using ginkgo bilo...
4,8,2021-07-28,Ginkgo,4.0,Goes to show you how unreliable WebMD is. This...


In [3]:
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

news['reviews'] = news['reviews'].fillna('')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lyuob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lyuob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lyuob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
import numpy as np
import nltk
from nltk.corpus import stopwords
import gensim
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim


def get_lda_objects(text):
    nltk.download('stopwords')    
    stop=set(stopwords.words('english'))

    
    def _preprocess_text(text):
        corpus=[]
        stem=PorterStemmer()
        lem=WordNetLemmatizer()
        for news in text:
            words=[w for w in word_tokenize(news) if (w not in stop)]

            words=[lem.lemmatize(w) for w in words if len(w)>2]

            corpus.append(words)
        return corpus
    
    corpus=_preprocess_text(text)
    
    dic=gensim.corpora.Dictionary(corpus)
    bow_corpus = [dic.doc2bow(doc) for doc in corpus]
    
    lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 4, 
                                   id2word = dic,                                    
                                   passes = 10,
                                   workers = 2)
    
    return lda_model, bow_corpus, dic

def plot_lda_vis(lda_model, bow_corpus, dic):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dic)
    return vis

In [5]:
lda_model, bow_corpus, dic = get_lda_objects(news['reviews'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lyuob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
lda_model.show_topics()

[(0,
  '0.017*"hair" + 0.013*"taking" + 0.012*"take" + 0.010*"week" + 0.009*"day" + 0.008*"month" + 0.008*"n\'t" + 0.007*"year" + 0.006*"level" + 0.006*"product"'),
 (1,
  '0.009*"take" + 0.008*"n\'t" + 0.008*"product" + 0.008*"brand" + 0.007*"day" + 0.007*"This" + 0.006*"taking" + 0.006*"one" + 0.005*"year" + 0.005*"time"'),
 (2,
  '0.008*"take" + 0.008*"time" + 0.007*"year" + 0.006*"silver" + 0.006*"use" + 0.006*"used" + 0.006*"n\'t" + 0.006*"sleep" + 0.005*"day" + 0.005*"taking"'),
 (3,
  '0.014*"taking" + 0.013*"day" + 0.008*"pain" + 0.008*"good" + 0.007*"effect" + 0.007*"started" + 0.006*"take" + 0.006*"sleep" + 0.005*"oil" + 0.005*"side"')]

In [7]:
plot_lda_vis(lda_model, bow_corpus, dic)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.070061 -0.036781       1        1  41.885001
3     -0.000016 -0.049851       2        1  24.266617
1     -0.027603  0.088120       3        1  19.934513
2      0.097680 -0.001488       4        1  13.913869, topic_info=         Term        Freq       Total Category  logprob  loglift
723      hair  354.000000  354.000000  Default  30.0000  30.0000
3445   silver  110.000000  110.000000  Default  29.0000  29.0000
143     brand  121.000000  121.000000  Default  28.0000  28.0000
469     sleep  135.000000  135.000000  Default  27.0000  27.0000
2138     nail  156.000000  156.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
119    taking   30.445463  511.643393   Topic4  -5.3767  -0.8494
510     month   23.575187  264.046952   Topic4  -5.6325  -0.4436
649       oil   20.775079  165.755787   Topic4  -5.7589  -0.1045
0      effect   19.465653  189.929199   Topic4  -5.8240  -0.3057
195   product   18.944490  249.682624   Topic4  -5.8512  -0.6064

[343 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
129       1  0.596134    've
129       2  0.112782    've
129       3  0.193341    've
129       4  0.104726    've
574       1  0.353661    ...
...     ...       ...    ...
376       4  0.131080  would
124       1  0.479659   year
124       2  0.186926   year
124       3  0.176345   year
124       4  0.155184   year

[639 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3])